# Neuro-Symbolic AI

## RFT Dataset

### Contents

- Generate Dataset
- Perception Model
- Semantic Parser
- Program Executor


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import cv2
from img_utils import *
from data_generator import *

In [5]:
# comaprison to NS-VQA sample

(img, objects, relations, norelations) = build_sample()


In [9]:
objects

[(0, array([ 55, 179]), 'r', (40, 164, 70, 194)),
 (1, array([130,  31]), 'c', (115, 16, 145, 46)),
 (2, array([115, 154]), 'r', (100, 139, 130, 169)),
 (3, array([ 69, 103]), 'c', (54, 88, 84, 118)),
 (4, array([77, 48]), 'r', (62, 33, 92, 63)),
 (5, array([115, 192]), 'r', (100, 177, 130, 207))]

In [10]:
# Generate the dataset

# Dataset params
img_size = 224
size = 15 # the object size?
question_size = 10 # 6 for one-hot vector of color, 1 for question type, 3 for question subtype
q_type_idx = 6 # type of question
sub_q_type_idx = 7 # 
nb_questions = 10 # questions per image. 

# Possibles Answers : [yes, no, rectangle, circle, 1, 2, 3, 4, 5, 6]

colors = [
    (0,0,255), # red
    (0,255,0), # green
    (255,0,0), # blue
    (0,156,255), # orange
    (128,128,128), # gray (128,128,128)- not working, replaced with black
    (0,255,255) # yellow - appears same as orange
]

# RFT Same-Different Data Generator

In [11]:
# Build a sample

def build_image(colors, img_size, size):
    # choose two random colors
# for i in range(16):
    color1 = ()
    color2 = ()
    while color1 == color2:
        color1 = colors[random.randint(0,5)]
        color2 = colors[random.randint(0,5)]

    positions = [([25,25],[65,25]),([25,175],[65,175]),([160,25],[200,25]), ([160,175],[200,175])]
    random.shuffle(positions)

    img = np.ones((img_size,img_size,3)) * 255

    img = draw_sCsS(img, positions, color1, size = size)
    img = draw_sCdS(img, positions, color2, size = size)
    img = draw_dCsS(img, positions, color1, color2, size = size)
    img = draw_dCdS(img, positions, color1, color2, size = size)

    img = (img).astype('uint8')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img


In [12]:
# build_img_dataset with params colors, img_size, size
for i in range(16):
    img = build_image(colors, img_size, size)
    cv2.imwrite(f'rft_dataset/{i}.png', img)

# Biuld Questions

In [13]:
def fill_q_template(idx):
    question_types = [['the same color', 'the same shape'],['the same color','a different shape'],\
        ['a different color','the same shape'], ['a different color','a different shape']]
    
    order = [0,1]
    random.shuffle(order)
    return f'Which pair is {question_types[idx][order[0]]} and {question_types[idx][order[1]]}?'


In [14]:
questions = [fill_q_template(i) for i in [0,1,2,3]]
random.shuffle(questions)
questions

['Which pair is the same shape and a different color?',
 'Which pair is a different shape and the same color?',
 'Which pair is the same color and the same shape?',
 'Which pair is a different color and a different shape?']

In [ ]:
# Generate training dataset

